In [163]:
import json
from symmer.chemistry import MoleculeBuilder
import os
import numpy as np
from openfermion import FermionOperator
from symmer import PauliwordOp

In [6]:
notebook_dir = os.getcwd()
# notebook_dir =  os.path.dirname(cwd)
symmer_dir = os.path.dirname(notebook_dir)
test_dir = os.path.join(symmer_dir, 'tests')
ham_data_dir = os.path.join(test_dir, 'hamiltonian_data')

if not os.path.isdir(ham_data_dir):
    raise ValueError('cannot find data dir')
    
filename = 'H4_STO-3G_SINGLET_JW.json'

In [9]:
ham_data_dir

'/Users/lex/Documents/PhD/symmer/tests/hamiltonian_data'

In [19]:
speciesname = 'HCl_STO-3G_SINGLET_JW.json'

with open(os.path.join(ham_data_dir, speciesname), 'r') as infile:
    molecule_geometries = json.load(infile)


# # build the molecule
mol_data = molecule_geometries['data']
geometry  = mol_data['geometry']
charge = mol_data['charge']
basis = mol_data['basis']
spin = 0


In [89]:
molecule = MoleculeBuilder(geometry=geometry,
                           charge=charge, 
                           basis=basis,
                           spin=spin, 
                           run_fci=False, 
                           run_mp2=False,
                           run_cisd=False,
#                            run_ccsd=False,
                           print_info=True,
                          symmetry='True')

Molecule geometry:
Cl	0.0	0.0	0.07452
H	0.0	0.0	-1.266846

FCI converged?  True

HF energy:   -455.1354456708121
MP2 energy:  None
CCSD energy: -455.1570668271786
CISD energy: None
FCI energy:  None


Number of qubits: 20


In [90]:
molecule.pyscf_obj.pyscf_hf

In [91]:
groupname = molecule.pyscf_obj.pyscf_hf.mol.groupname,
print(groupname)
topgroup=molecule.pyscf_obj.pyscf_hf.mol.topgroup
print(topgroup)

('C2v',)
Coov


In [92]:
from pyscf import symm


orbsym = symm.label_orb_symm(molecule.pyscf_obj.pyscf_hf.mol,
                             molecule.pyscf_obj.pyscf_hf.mol.irrep_name,
                             molecule.pyscf_obj.pyscf_hf.mol.symm_orb,
                             molecule.pyscf_obj.pyscf_hf.mo_coeff)

In [93]:
# orb_sym_labels = np.zeros((len(orbsym)*2), dtype=object)

# orb_sym_labels[0::2] = orbsym
# orb_sym_labels[1::2] = orbsym

In [94]:
labels_alpha = orbsym[np.where(molecule.hf_array[::2])]
labels_beta = orbsym[np.where(molecule.hf_array[1::2])]

labels = np.dstack((labels_alpha,labels_beta)).flatten()
labels

array(['A1', 'A1', 'A1', 'A1', 'A1', 'A1', 'B1', 'B1', 'B2', 'B2', 'A1',
       'A1', 'A1', 'A1', 'B1', 'B1', 'B2', 'B2'], dtype='<U2')

In [95]:
# note can only have values of +1 or -1 here!
C2 = {'A1': 1,
      'A2': 1,
      'B1':-1,
      'B2':-1,}

sig_h = { 'A1': 1,
          'A2':-1,
          'B1': 1,
          'B2':-1,}



In [96]:
labels

array(['A1', 'A1', 'A1', 'A1', 'A1', 'A1', 'B1', 'B1', 'B2', 'B2', 'A1',
       'A1', 'A1', 'A1', 'B1', 'B1', 'B2', 'B2'], dtype='<U2')

In [137]:
ferms = []
symm_dict = sig_h # [C2 or sig_h]
for qind, term in enumerate(labels):
    if term in symm_dict.keys():
        if symm_dict[term]==-1:

            op = FermionOperator(f'',1) + FermionOperator(f'{qind}^ {qind}', -2)
            ferms.append(op)

In [138]:
from functools import reduce
from openfermion import jordan_wigner


sig_h_sym_op_ferm = reduce(lambda x,y:x+y, ferms)

jordan_wigner(sig_h_sym_op_ferm)



(1+0j) [Z8] +
(1+0j) [Z9] +
(1+0j) [Z16] +
(1+0j) [Z17]

http://gernot-katzers-spice-pages.com/character_tables/C2v.html

In [139]:
from symmer.symplectic import IndependentOp

stabs = IndependentOp.symmetry_generators(molecule.H_q, commuting_override=True)
stabs

 1 IIIIIIZZIIIIIIZZIIII 
 1 IIIIIIIIZZIIIIIIZZII 
 1 ZIZIZIIZIZZIZIIZIZZI 
 1 IZIZIZIZIZIZIZIZIZIZ

In [140]:
np.where(np.array(list(stabs[1].__str__()[3:]))!='I')

(array([ 8,  9, 16, 17]),)

In [149]:
ferms = []
symm_dict = C2 # [C2 or sig_h]
for qind, term in enumerate(labels):
    if term in symm_dict.keys():
        if symm_dict[term]==-1:

            op = FermionOperator(f'',1) + FermionOperator(f'{qind}^ {qind}', -2)
            ferms.append(op)
            
            
C2_sym_op_ferm = reduce(lambda x,y:x+y, ferms)

jordan_wigner(C2_sym_op_ferm)

(1+0j) [Z6] +
(1+0j) [Z7] +
(1+0j) [Z8] +
(1+0j) [Z9] +
(1+0j) [Z14] +
(1+0j) [Z15] +
(1+0j) [Z16] +
(1+0j) [Z17]

In [142]:
np.where(np.array(list(stabs[0].__str__()[3:]))!='I')

(array([ 6,  7, 14, 15]),)

In [145]:
np.where(np.array(list(stabs[3].__str__()[3:]))!='I')

(array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),)

In [162]:
sigH = list('I'*stabs.n_qubits)

sigH[8]= 'Z'
sigH[9]= 'Z'
sigH[16]= 'Z'
sigH[17]= 'Z'

PauliwordOp.from_list([''.join(sigH)])#.to_openfermion

(1+0j) [Z8 Z9 Z16 Z17]

In [164]:
sigH = list('I'*stabs.n_qubits)

sigH[8]= 'Z'
sigH[9]= 'Z'
sigH[16]= 'Z'
sigH[17]= 'Z'

sigH = PauliwordOp.from_list([''.join(sigH)])#.to_openfermion


In [168]:


C2_term = list('I'*stabs.n_qubits)

C2_term[6]= 'Z'
C2_term[7]= 'Z'
C2_term[8]= 'Z'
C2_term[9]= 'Z'
C2_term[14]= 'Z'
C2_term[15]= 'Z'
C2_term[16]= 'Z'
C2_term[17]= 'Z'

C2_term = PauliwordOp.from_list([''.join(C2_term)])
C2_term

 1.000+0.000j IIIIIIZZZZIIIIZZZZII

In [170]:
symm_ops = sigH + C2_term

In [172]:
symm_ops.basis_reconstruction(stabs)

(array([[0, 1, 0, 0],
        [1, 1, 0, 0]]),
 array([ True,  True]))

In [169]:
# 'Z8 Z9 Z16 Z17' (sigma_h symmetry)

# 'Z6 Z7 Z8 Z9 Z14 Z15 Z16 Z17' (C2 symmetry)